In [1]:
# imports
# -------

import numpy as np
import random
import copy
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import csv
import cv2
import h5py
import time
from tqdm import tqdm
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
import os
from os import listdir
from os.path import isfile, join
from PIL import Image
import pandas as pd
import multiprocessing
from scipy.ndimage import label
from skimage import measure

# torch related imports
# ---------------------
import torch
from torch.autograd import Variable
import torch.autograd
import torch.nn as nn
import torch.nn.functional as F


# RNN related
# -----------
import nltk
import gensim
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models import Word2Vec
from torch.nn.utils.rnn import pad_sequence


# local settings
# --------------
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")

%matplotlib inline
%env JOBLIB_TEMP_FOLDER=/tmp


/Users/venkateshmadhava/Documents/pmate2/pmate2_env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


env: JOBLIB_TEMP_FOLDER=/tmp


# 1. Code

### 1.1 helper functions

In [2]:
# cosine similarity function to find similarity distance between two words
# ------------------------------------------------------------------------

def cosine_similarity(u, v):
    
    """
    Cosine similarity reflects the degree of similariy between u and v
        
    Arguments:
        u -- a word vector of shape (n,)          
        v -- a word vector of shape (n,)

    Returns:
        formula -- cos thetha = dot_product_of(u,v)/l2_norm(u)*l2_norm(v)
        cosine_similarity -- the cosine similarity between u and v defined by the formula above.
    """
    
    
    # Compute the dot product between u and v (≈1 line)
    # -------------------------------------------------
    dot = np.dot(u,v)
    norm_u = np.sqrt(np.sum(np.square(u)))
    norm_v = np.sqrt(np.sum(np.square(v)))
    cosine_similarity = dot/(norm_u*norm_v)
    
    # final return
    # -----------
    return cosine_similarity

In [3]:
# function to prepare user input dataset
# --------------------------------------


def prepare_use_input(user_inlist, num_layers, num_directions, hidden_state_size):
    
    # 1. prepare user input
    # ---------------------
    print('1. preparing user input..')
    userin_dict = {}
    for i in range(len(user_inlist)):
        temp_list = []
        userin_dict[i] = {}
        for each in user_inlist[i].split(' '):
            temp_list.append(''.join(ch for ch in each.lower() if ch.isalnum()))
        userin_dict[i]['words'] = temp_list
        userin_dict[i]['label'] = -1
        
        
    # creating input dataset
    # ----------------------
    print('2. Setting up datasets..')
    xin_user,yin_user = create_rnn_input_from_indict(userin_dict)
    print('##')
    print(xin_user.size())
    print(xin_user.type())
    print(yin_user.size())
    print(yin_user.type())
    h_0_user = torch.zeros((num_layers*num_directions, xin_user.size()[1], hidden_state_size))
    print(h_0_user.size())
    
    return xin_user, h_0_user
    

In [4]:
# Function to build a dataset from input text
# -------------------------------------------

def build_sentence_in_dataset(file_url):
    
    '''
    
    1. takes in txt file with sentence_/t_label_/n format
    2. outputs dict of format dict[index]['words'] = list of words in sentence, dict[index]['label'] = target
    
    '''
    
    # 0. initialisations
    # ------------------
    x_dict = {}
    counter = 0
    total = 0
    not_read = []

    # 1. file open ops
    # ----------------
    file_in = open(file_url,'r')
    file_in_list = file_in.read().split('\n')

    # 2. list processing into dict ops
    # --------------------------------
    for i in range(len(file_in_list)):

        # flag update
        # -----------
        total += 1

        # main statements
        # ---------------
        try:
            curr_data = file_in_list[i]
            curr_label = int(curr_data.split('\t')[1])
            curr_sentence = curr_data.split('\t')[0].lower()
            curr_sentence_words_alnum = []
            
            # keeping only alpha numeric chars
            # --------------------------------
            for each in curr_sentence.split(' '):
                curr_sentence_words_alnum.append(''.join(ch for ch in each if ch.isalnum()))
                

            # saving to dict
            # --------------
            x_dict[i] = {}
            x_dict[i]['words'] = curr_sentence_words_alnum
            x_dict[i]['label'] = curr_label
            counter += 1
        except:
            not_read.append(curr_data)


    # 2.1 info print
    # --------------
    print(str(counter) + ' lines processed of a total of ' + str(total) )
    
    
    # final return
    # ------------
    return x_dict

In [5]:
# function that builds a RNN input using input dict
# --------------------------------------------------

def create_rnn_input_from_indict(indict):
    
    '''
    
    1. Takes input dict of format dict[index]['words'] = list of words in sentence, dict[index]['label'] = target
    2. outputs RNN input of format (seq_len, batch, input_size)
    3. That is (no_words_per_sentence, no_example_sentences, vector_len_of_each_word)
    4. outputs target label (no_example_sentences) -- NOT ONE HOT
    
    '''
    
    # 0. initialisations
    # ------------------
    feature_vec_size = word2vec_dict['hi'].shape[0]
    xout_list = []
    target = np.zeros((0))
    
    
    # 1. looping through dict
    # -----------------------
    print('1. Itering through in dict..')
    for keys in indict:
        
        # 1.0 initialising temp seq for this current sentence
        # ---------------------------------------------------
        temp_seq = np.zeros((0,feature_vec_size))
        
        # looping through words
        # ---------------------
        for each in indict[keys]['words']:

            # 1.1 checking if word2vec has the word
            # -------------------------------------
            try:
                curr_wordvec = word2vec_dict[each].reshape(1,feature_vec_size)
            except:
                curr_wordvec = np.zeros((1,feature_vec_size))

            # 1.2 building sequence
            # ---------------------
            temp_seq = np.concatenate((temp_seq,curr_wordvec), axis = 0)
            
        
        # concatenating to output
        # -----------------------
        xout_list.append(torch.from_numpy(temp_seq))
        target = np.concatenate((target,np.array([indict[keys]['label']])))
    
    
    # 2. final padding before return
    # ------------------------------
    print('2. Padding list..')
    xout = pad_sequence(xout_list)
    target = torch.from_numpy(target)
    target = target.view(-1,1)
    
    # final return
    # ------------
    return xout.float(), target.float()
    

### 1.2 NN related functions

In [6]:
# GENERIC - initialises weights for a NN
# --------------------------------------
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
    #    print(classname)
        m.weight.data.normal_(0.0, 0.02)
    #elif classname.find('Linear') != -1:
    #    print(classname)
    #    m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)
        
        
# GENERIC class that inherits nn module and makes a sequential object a model
# ---------------------------------------------------------------------------
class Net(nn.Module):
    def __init__(self,sequencelist):
        super().__init__() # Initializing nn.Module construtors
        self.forwardpass = sequencelist
        
    def forward(self,x):
        xout = self.forwardpass(x)
        return xout

In [7]:
# Function to build a lineaf FC model
# -----------------------------------

def linear_fc(layers, nw_activations, target_activation, dropout_p):
    
    'The first value in the layers list is the input dimensions of the input'
    
    # 0. initialisations
    # ------------------
    seq_list = []
    
    # setting N/W activations
    # -------------------
    if nw_activations == 'relu':
        nw_act = nn.ReLU()
    elif nw_activations == 'lrelu':
        nw_act = nn.LeakyReLU(0.2)
    elif nw_activations == 'sigmoid':
        nw_act = nn.Sigmoid()
    elif nw_activations == 'tanh':
        nw_act = nn.Tanh()
    else:
        nw_act = nn.ReLU()
    
    # setting target activations
    # --------------------------
    if target_activation == 'sigmoid':
        target_act = nn.Sigmoid()
    elif target_activation == 'softmax':
        target_act = nn.Softmax()
    else:
        target_act = None
    
    # 1. building n/w's layer list
    # ----------------------------
    network = []
    for k in range(len(layers)):
        try:
            network.append((layers[k],layers[k+1]))
        except:
            pass
            

    
    # 2. constructing encoder n/w
    # ----------------------------
    for i in range(len(network)):
        
        # 2.1 adding linear layers to encoder
        # ------------------------------------
        curr_dims = network[i]
        seq_mod = nn.Linear(curr_dims[0],curr_dims[1])
        seq_list.append(seq_mod)
        
        # checking last layer or not
        # --------------------------
        if i+1 == len(network):
            
            # at last layer
            # -------------
            if target_act == None:
                pass
            else:
                seq_list.append(target_act)
        
        else:
            
            # batchnorm
            # ---------
            seq_list.append(nn.BatchNorm1d(curr_dims[1]))
          
            # non linear activation
            # ---------------------
            seq_list.append(nw_act)
            
            # dropout
            # -------
            seq_list.append(nn.Dropout(p = dropout_p))
           
            
    
    # 3. returning model
    # ------------------
    seq_list = nn.Sequential(*seq_list)
    seq_list.apply(weights_init)

    model = Net(seq_list)
    model = model.train()
    
    return model
            
            


In [8]:
# a function to load a saved model
# --------------------------------

def load_saved_model_function(in_path, mode, use_cuda):
    
    
    '''
    
    path = /folder1/folder2/model_sentiment.tar format 
    
    There will be two or more models associated with this depending on the task -
    model_sentiment_rnn, model_sentiment_dense
    
    '''
    
    # 0. Initialisation
    # ------------------
    models_out = {}
    
    if mode == 'rnn_dense':
    
        # 1. loading RNN model
        # --------------------
        rnn_path = in_path.replace('.tar','_RNN.tar')
        model = torch.load(in_path.replace('.tar','_RNN_MODEL.tar'))
        optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad,model.parameters()))

        # Applying state dict
        # ----------------------
        if use_cuda == True:
            
            # loads to GPU
            # ------------
            checkpoint = torch.load(rnn_path)
            
        else:
            
            # loads to CPU
            # ------------
            checkpoint = torch.load(rnn_path, map_location=lambda storage, loc: storage)

        # loading sate dict values
        # ---------------------------
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch']
        
        # building list
        # -------------
        models_out['rnn'] = [model, optimizer, epoch]
        
        
        # 1. loading Dense model
        # --------------------
        dense_path = in_path.replace('.tar','_DENSE.tar')
        model = torch.load(in_path.replace('.tar','_DENSE_MODEL.tar'))
        optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad,model.parameters()))

        # Applying state dict
        # ----------------------
        if use_cuda == True:
            # loads to GPU
            # ------------
            checkpoint = torch.load(dense_path)
        else:
            # loads to CPU
            # ------------
            checkpoint = torch.load(dense_path, map_location=lambda storage, loc: storage)

        # loading sate dict values
        # ---------------------------
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch']
        loss = checkpoint['loss']
        loss_mode = checkpoint['loss_mode']
        
        # building list
        # -------------
        models_out['dense'] = [model, optimizer, epoch, loss, loss_mode]
        
    else:
        
        assert 1 == 2, 'Other modes not built in yet.'
    
    
    # final return
    # ------------
    return models_out
    

In [9]:
# GENERIC model function to train the networks
# --------------------------------------------

def model_rnn_train(xin,hin,yin,xval,hval,yval,load_mode,model_mode,model_dict,epochs,mbsize,loss_mode,use_cuda,save_state,path):
    
    # 0. initialisations
    # ------------------
    loss_train = []
    loss_val = []
    models_return_out = {}
    

    # 1. loading old model if required
    # --------------------------------
    if load_mode == 'from saved':
        
        # 0. loading saved models
        # -----------------------
        model_dict = load_saved_model_function(path,model_mode,use_cuda)
        
        # checking model modes and loading them
        # --------------------------------------
        if model_mode == 'rnn_dense':
            

            # 1. loading RNN from saved
            # -------------------------
            model_rnn = model_dict['rnn'][0]
            optimizer_rnn = model_dict['rnn'][1]
            saved_epoch_rnn = model_dict['rnn'][2]
            
            model_rnn = model_rnn.train()
            print('Loading RNN model from saved state...')
            print('Last saved epoch - ' + str(saved_epoch_rnn))
           
            
            # 2. loading dense net from saved
            # -------------------------------
            model_dense = model_dict['dense'][0]
            optimizer_dense = model_dict['dense'][1]
            saved_epoch_dense = model_dict['dense'][2]
            saved_loss_dense = model_dict['dense'][3]
            saved_loss_mode_dense = model_dict['dense'][4]
            
            model_dense = model_dense.train()
            dense_loss_mode = saved_loss_mode_dense
            print('Loading Dense model from saved state...')
            print('Last saved epoch - ' + str(saved_epoch_dense))
            print('Last saved loss - ' + str(saved_loss_dense))
            
            
            # 3. sanity assertion
            # -------------------
            assert int(saved_epoch_rnn) == int(saved_epoch_dense), 'FATAL ERROR: Rnn and densenet have different training epochs.'
            epochs += int(saved_epoch_rnn)
            start_epoch = int(saved_epoch_rnn)
            
            models_return_out['rnn'] = model_rnn
            models_return_out['dense'] = model_dense
        
        else:
            
            assert 1==2, 'Other modes not coded yet.'
        
    else:
        
        # building new models
        # -------------------
        start_epoch = 1
        
        # checkimg model mode
        # -------------------
        if model_mode == 'rnn_dense':
            
            # extracting models
            # -----------------
            model_rnn = model_dict['rnn']
            model_rnn = model_rnn.train()
            optimizer_rnn = torch.optim.Adam(filter(lambda p: p.requires_grad,model_rnn.parameters()))
        
            model_dense = model_dict['dense']
            model_dense = model_dense.train()
            optimizer_dense = torch.optim.Adam(filter(lambda p: p.requires_grad,model_dense.parameters()))
            
            models_return_out['rnn'] = model_rnn
            models_return_out['dense'] = model_dense
        
        
        else:
            
            assert 1==2, 'Other modes not coded yet.'
            
        
    
    # 2. setting loss criterion
    # -------------------------
    if loss_mode == 'MSE':
        criterion = nn.MSELoss()
    elif loss_mode == 'BCE':
        criterion = nn.BCELoss()
    elif loss_mode == 'NLL':
        criterion = nn.NLLLoss()
    elif loss_mode == 'crossentropy':
        criterion = nn.CrossEntropyLoss()
        yin = torch.max(yin.long(),1)[1]
    else:
        criterion = nn.MSELoss()
        
        
    
    # 3. Setting up minibatch features
    # --------------------------------
    m = xin.size()[1]
    mb_list = []
    mb_list = list(range(int(m/mbsize)))
    if m % mbsize == 0: # if the minibatches can be split up perfectly.
        'do nothing'
    else:
        mb_list.append(mb_list[len(mb_list)-1] + 1)
    
    
    # 4. ACTUAL TRAINING ITERATIONS
    # -----------------------------
    if model_mode == 'rnn_dense':
    
        # 4.1 training iters
        # ------------------
        for i in range(start_epoch,epochs+1):

            for p in mb_list:

                # Mini batch operations
                # ---------------------
                start_index = p*mbsize
                end_index = m if p == mb_list[len(mb_list)-1] else p*mbsize + mbsize
                m_curr = end_index - start_index

                Xin_mb = xin[:,start_index:end_index,:]
                Hin_mb = hin[:,start_index:end_index,:]
                Yin_mb = yin[start_index:end_index]

                if use_cuda == True:
                    torch.cuda.empty_cache()
                    model_rnn = model_rnn.cuda()
                    model_dense = model_dense.cuda()
                    Xin_mb = Xin_mb.cuda()
                    Yin_mb = Yin_mb.cuda()
                    Hin_mb = Hin_mb.cuda()

                
                # RNN network ops
                # ---------------
                _,model_rnn_out = model_rnn(Xin_mb,Hin_mb)
                #print(model_rnn_out.size())
                model_rnn_out = model_rnn_out.permute(1,0,2)
                model_rnn_out = model_rnn_out.view(model_rnn_out.size()[0],-1)
                #print(model_rnn_out.size())
              
                
                # Dense NN network ops
                # --------------------
                model_out = model_dense(model_rnn_out)
                #print(model_out.size())
                #print(Yin_mb.size())
                
                # backward ops
                # ------------
                optimizer_rnn.zero_grad()
                optimizer_dense.zero_grad()
                loss = criterion(model_out.float(), Yin_mb.float()) # loss(output, target)
                loss.backward()
                optimizer_rnn.step()
                optimizer_dense.step()
                loss_train.append(loss.item())

                # deleting curr variables
                # -----------------------
                if use_cuda == True:
                    Xin_mb = Xin_mb.cpu()
                    Yin_mb = Yin_mb.cpu()
                    Hin_mb = Hin_mb.cpu()
                    model_out = model_out.cpu()

                    del Xin_mb
                    del Yin_mb
                    del Hin_mb
                    del model_out
                    torch.cuda.empty_cache()

                
                # printing loss
                # -------------
                print('Epoch ' + str(i) + ', minibatch ' + str(p+1) + ' of '  +  str(len(mb_list)) + ' -- Model loss: ' + str(loss.item()))

        
        # 4.1.1 preparing function out dict
        # ---------------------------------
        models_return_out['rnn'] = model_rnn
        models_return_out['dense'] = model_dense
        
        
        # 4.2 outside training loop saving model states
        # --------------------------------------------
        if save_state == True and epochs+1 > start_epoch:

            # 4.2.1 initialising rnn save dict
            # -------------------------------
            save_dict = {}
            save_dict['epoch'] = str(i)
            save_dict['model_state_dict'] = model_rnn.cpu().state_dict()
            save_dict['optimizer_state_dict'] = optimizer_rnn.state_dict()
            
            # 4.2.2 saving
            # ----------
            rnn_path = path.replace('.tar','_RNN.tar')
            torch.save(save_dict,rnn_path)

            # saving full model to initialise a new model later on
            # ----------------------------------------------------
            torch.save(model_rnn.cpu(),rnn_path.replace('.tar','_MODEL.tar'))
            print('Saved RNN net.')
            
            
            # 4.2.3 saving dense NN state
            # ---------------------------
            save_dict = {}
            save_dict['epoch'] = str(i)
            save_dict['model_state_dict'] = model_dense.cpu().state_dict()
            save_dict['optimizer_state_dict'] = optimizer_dense.state_dict()
            save_dict['loss'] = str(loss.item())
            save_dict['loss_mode'] = loss_mode
            
             
            # 4.2.4 saving
            # ----------
            dense_path = path.replace('.tar','_DENSE.tar')
            torch.save(save_dict,dense_path)

            # saving full model to initialise a new model later on
            # ----------------------------------------------------
            torch.save(model_dense.cpu(),dense_path.replace('.tar','_MODEL.tar'))
            
            print('Saved dense net.')
    
    else:
        
        assert 1==2, 'Other modes not coded yet.'
        
    
    

    # 4. return model in order to use elsewhere in the code
    # -----------------------------------------------------
    return models_return_out
        


In [10]:
# inference code
# --------------

def inference(xin,hin,mode,model_dict,use_cuda):
    
    # 0. initialisations
    # ------------------
    # NONE
    
    # 1. check mode and forward pass ops
    # ----------------------------------
    if mode == 'rnn_dense':
        
        # 1.0 setting up n/w
        # ------------------
        model_rnn = model_dict['rnn']
        model_dense = model_dict['dense']
        
        model_rnn = model_rnn.eval()
        model_dense = model_dense.eval()
        
        

        # 1.1 Setting up minibatch features
        # --------------------------------
        m = xin.size()[1]
        mbsize = 3
        mb_list = []
        mb_list = list(range(int(m/mbsize)))
        if m % mbsize == 0: # if the minibatches can be split up perfectly.
            'do nothing'
        else:
            mb_list.append(mb_list[len(mb_list)-1] + 1)
            
            
        # 1.2 Mini batch ops
        # ------------------
        with tqdm(total=m) as pbar:
            for p in mb_list:

                # Mini batch operations
                # ---------------------
                start_index = p*mbsize
                end_index = m if p == mb_list[len(mb_list)-1] else p*mbsize + mbsize
                m_curr = end_index - start_index

                Xin_mb = xin[:,start_index:end_index,:]
                Hin_mb = hin[:,start_index:end_index,:]
                Yin_mb = yin[start_index:end_index]

                if use_cuda == True:
                    torch.cuda.empty_cache()
                    model_rnn = model_rnn.cuda()
                    model_dense = model_dense.cuda()
                    Xin_mb = Xin_mb.cuda()
                    Yin_mb = Yin_mb.cuda()
                    Hin_mb = Hin_mb.cuda()


                # RNN network ops
                # ---------------
                _,model_rnn_out = model_rnn(Xin_mb,Hin_mb)
                model_rnn_out = model_rnn_out.permute(1,0,2)
                model_rnn_out = model_rnn_out.view(model_rnn_out.size()[0],-1)

                # Dense NN network ops
                # --------------------
                model_out = model_dense(model_rnn_out)

                # concat opts
                # -----------
                try:
                    final_out = torch.cat((final_out,model_out), 0)
                except:
                    final_out = model_out
            
                # pbar update
                # -----------
                pbar.update(mbsize)


    else:
        assert 1==2, 'Other modes not coded yet.'
        
    
    # 2. final return
    # ---------------
    return final_out.data.cpu().numpy()
    
    
    

### 1.3 NN classes

In [11]:
class RNN_NW(nn.Module):
    
    def __init__(self, mode, input_size, hidden_size, num_layers, bidirectional):
        super().__init__()
        
        # 0. initialisationing n/w
        # ------------------------
        if mode == 'gru':
            self.rnn = nn.GRU(input_size=input_size, hidden_size=hidden_size*num_layers, num_layers=num_layers, bidirectional=bidirectional)
        else:
            self.rnn = nn.LSTM(input_size=input_size, hidden_size=hidden_size*num_layers, num_layers=num_layers, bidirectional=bidirectional)

    
    
    
    def forward(self, input, hidden):
        
        
        '''
        
        Outputs: output, h_n
        --------------------
        output of shape (seq_len, batch, num_directions * hidden_size): tensor containing the output features h_t from the last layer of the GRU, for each t. If a torch.nn.utils.rnn.PackedSequence has been given as the input, the output will also be a packed sequence. For the unpacked case, the directions can be separated using output.view(seq_len, batch, num_directions, hidden_size), with forward and backward being direction 0 and 1 respectively.
        Similarly, the directions can be separated in the packed case.
        
        output - the hidden state values for every word of every sentence example -- for each t
        use this for word wise predictions
        example - torch.Size([32, 1000, 128])

        h_n of shape (num_layers * num_directions, batch, hidden_size): tensor containing the hidden state for t = seq_len
        Like output, the layers can be separated using h_n.view(num_layers, num_directions, batch, hidden_size).
        
        hn - the hidden state values for every sentence example at last words onlye -- for time step t = seq_len
        use this for sentence wise prediction
        example - torch.Size([1, 1000, 128])
        
        '''

        output, hidden = self.rnn.double()(input.double(), hidden.double())
        return output.float(), hidden.float()


# 2. Running of codes

### 2.1 setting up env variables

In [12]:
## SET UP CUDA OR NOT HERE + OTHER SET UPS
##########################################

dev_env = 'local' # 'gpu' or 'local'

##########################################
##########################################


# Setting CUDA
# ------------
if dev_env == 'gpu':
    use_cuda = True
else:
    use_cuda = False
if use_cuda == True:
    torch.cuda.empty_cache()
    


# SET FILE SPECIFIC NAMES HERE
# ----------------------------
if dev_env == 'gpu':
    save_path = '/home/venkateshmadhava/codes/pmate2_localgpuenv/models/'
    parent_url = '/home/venkateshmadhava/datasets/images'
else:
    save_path = '/Users/venkateshmadhava/Documents/pmate2/pmate2_env/models/'
    parent_url = '/Users/venkateshmadhava/Documents/projects/sequence/sentiment_labelled_sentences'


# displaying model path
# --------------------
print(save_path)

/Users/venkateshmadhava/Documents/pmate2/pmate2_env/models/


In [14]:
# loading word2vec model
# ----------------------

#word2vec_dict = gensim.models.KeyedVectors.load_word2vec_format(save_path + 'GoogleNews-vectors-negative300.bin', binary=True)  

In [15]:
# Showing some stats of the model
# -------------------------------

print('Total words in the vocab is ' + str(len(word2vec_dict.vocab)))
print('The dimension of each vector is ' + str(word2vec_dict['father'].shape[0]))

Total words in the vocab is 3000000
The dimension of each vector is 300


In [16]:
# checking similarities
# ---------------------

word_pairs = [('jupiter','earth'),('king','queen'),('inexpensive','cheap'),('mice','rat')]

for each in word_pairs:
    print('Similarity between ' + each[0] + ' and ' + each[1] + ' is: ' + str(cosine_similarity(word2vec_dict[each[0]],word2vec_dict[each[1]])))

Similarity between jupiter and earth is: 0.31955674
Similarity between king and queen is: 0.65109557
Similarity between inexpensive and cheap is: 0.70098954
Similarity between mice and rat is: 0.49135464


### 2.2 main calls

### 2.2.1 setting up data

In [19]:
# importing dataset
# -----------------

fileurl = parent_url + '/yelp_labelled.txt'
xdict = build_sentence_in_dataset(fileurl)

1000 lines processed of a total of 1001


In [20]:
# sanity viewing of data
# ----------------------

print(xdict[919])
print(xdict[325])
print(xdict[502])

{'label': 0, 'words': ['what', 'should', 'have', 'been', 'a', 'hilarious', 'yummy', 'christmas', 'eve', 'dinner', 'to', 'remember', 'was', 'the', 'biggest', 'fail', 'of', 'the', 'entire', 'trip', 'for', 'us']}
{'label': 1, 'words': ['the', 'black', 'eyed', 'peas', 'and', 'sweet', 'potatoes', 'unreal']}
{'label': 1, 'words': ['cant', 'say', 'enough', 'good', 'things', 'about', 'this', 'place']}


In [21]:
# creating input dataset
# ----------------------

xin,yin = create_rnn_input_from_indict(xdict)
print('##')
print(xin.size())
print(xin.type())
print(yin.size())
print(yin.type())

1. Itering through in dict..
2. Padding list..
##
torch.Size([32, 1000, 300])
torch.FloatTensor
torch.Size([1000, 1])
torch.FloatTensor


### 2.2.3 training model

In [44]:
# Setting up RNN hyper params
# ---------------------------

seq_vec_size = word2vec_dict['hi'].shape[0]
hidden_state_size = 32
num_layers = 1
bidirectional = False
rnn_mode = 'gru'

# initialising h_0 with shape (num_layers * num_directions, batch, hidden_size)
# -----------------------------------------------------------------------------
if bidirectional == True:
    num_directions = 2
else:
    num_directions = 1
h_0 = torch.zeros((num_layers*num_directions, xin.size()[1], hidden_state_size))
h_0.size()

torch.Size([1, 1000, 32])

In [45]:
# Setting up RNN
# -------------

' SETTING UP THE RNN '

model_rnn_senti = RNN_NW(rnn_mode,seq_vec_size,hidden_state_size,num_layers,bidirectional)
model_rnn_senti.apply(weights_init)

RNN_NW(
  (rnn): GRU(300, 32)
)

In [47]:
# Setting up fully connected dense NN
# -----------------------------------

' SETTING UP THE DENSE NN '

target_main_model_act = 'sigmoid'
dense_layers = [h_0.size()[2],16,1]
model_dense_senti = linear_fc(dense_layers, 'relu', target_main_model_act, 0.2)
model_dense_senti

Net(
  (forwardpass): Sequential(
    (0): Linear(in_features=32, out_features=16, bias=True)
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=16, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

In [48]:
# Training model dict
# -------------------

training_models_dict = {}
training_models_dict['rnn'] = model_rnn_senti
training_models_dict['dense'] = model_dense_senti


In [49]:
# training the models
# ------------------

''' USE -1 AS EPOCHS TO LOAD SAVED MODEL WITHOUT TRAINING '''

# model_rnn_train(xin,yin,xval,yval,load_mode,model_mode,model_dict,epochs,mbsize,loss_mode,use_cuda,save_state,path)

cn_file_name = 'model_rnndense_sentiment.tar'
cn_save_path = save_path + cn_file_name
print(cn_save_path)


md = model_rnn_train(xin,h_0,yin,None,None,None,'new','rnn_dense',training_models_dict,50,64,'MSE',use_cuda,True,cn_save_path)

/Users/venkateshmadhava/Documents/pmate2/pmate2_env/models/model_rnndense_sentiment.tar
Epoch 1, minibatch 1 of 16 -- Model loss: 0.2479955554008484
Epoch 1, minibatch 2 of 16 -- Model loss: 0.2546471357345581
Epoch 1, minibatch 3 of 16 -- Model loss: 0.2504749596118927
Epoch 1, minibatch 4 of 16 -- Model loss: 0.24182291328907013
Epoch 1, minibatch 5 of 16 -- Model loss: 0.24123400449752808
Epoch 1, minibatch 6 of 16 -- Model loss: 0.24330924451351166
Epoch 1, minibatch 7 of 16 -- Model loss: 0.2597361207008362
Epoch 1, minibatch 8 of 16 -- Model loss: 0.24123038351535797
Epoch 1, minibatch 9 of 16 -- Model loss: 0.25934499502182007
Epoch 1, minibatch 10 of 16 -- Model loss: 0.25601711869239807
Epoch 1, minibatch 11 of 16 -- Model loss: 0.24160785973072052
Epoch 1, minibatch 12 of 16 -- Model loss: 0.24975517392158508
Epoch 1, minibatch 13 of 16 -- Model loss: 0.2624039053916931
Epoch 1, minibatch 14 of 16 -- Model loss: 0.2433844953775406
Epoch 1, minibatch 15 of 16 -- Model loss: 0.

Epoch 9, minibatch 5 of 16 -- Model loss: 0.2099061757326126
Epoch 9, minibatch 6 of 16 -- Model loss: 0.1925184279680252
Epoch 9, minibatch 7 of 16 -- Model loss: 0.19410356879234314
Epoch 9, minibatch 8 of 16 -- Model loss: 0.21048276126384735
Epoch 9, minibatch 9 of 16 -- Model loss: 0.21426492929458618
Epoch 9, minibatch 10 of 16 -- Model loss: 0.22104546427726746
Epoch 9, minibatch 11 of 16 -- Model loss: 0.18635447323322296
Epoch 9, minibatch 12 of 16 -- Model loss: 0.20665013790130615
Epoch 9, minibatch 13 of 16 -- Model loss: 0.18766018748283386
Epoch 9, minibatch 14 of 16 -- Model loss: 0.1722354292869568
Epoch 9, minibatch 15 of 16 -- Model loss: 0.20584087073802948
Epoch 9, minibatch 16 of 16 -- Model loss: 0.23977099359035492
Epoch 10, minibatch 1 of 16 -- Model loss: 0.18279124796390533
Epoch 10, minibatch 2 of 16 -- Model loss: 0.16751131415367126
Epoch 10, minibatch 3 of 16 -- Model loss: 0.15284226834774017
Epoch 10, minibatch 4 of 16 -- Model loss: 0.14556549489498138


Epoch 17, minibatch 8 of 16 -- Model loss: 0.079726941883564
Epoch 17, minibatch 9 of 16 -- Model loss: 0.0605379194021225
Epoch 17, minibatch 10 of 16 -- Model loss: 0.05978836491703987
Epoch 17, minibatch 11 of 16 -- Model loss: 0.0632249042391777
Epoch 17, minibatch 12 of 16 -- Model loss: 0.08553969860076904
Epoch 17, minibatch 13 of 16 -- Model loss: 0.08979213237762451
Epoch 17, minibatch 14 of 16 -- Model loss: 0.07081244140863419
Epoch 17, minibatch 15 of 16 -- Model loss: 0.19838890433311462
Epoch 17, minibatch 16 of 16 -- Model loss: 0.2599713206291199
Epoch 18, minibatch 1 of 16 -- Model loss: 0.05536564812064171
Epoch 18, minibatch 2 of 16 -- Model loss: 0.06470775604248047
Epoch 18, minibatch 3 of 16 -- Model loss: 0.07366941124200821
Epoch 18, minibatch 4 of 16 -- Model loss: 0.04658177122473717
Epoch 18, minibatch 5 of 16 -- Model loss: 0.08584412187337875
Epoch 18, minibatch 6 of 16 -- Model loss: 0.06748919188976288
Epoch 18, minibatch 7 of 16 -- Model loss: 0.08483895

Epoch 25, minibatch 15 of 16 -- Model loss: 0.12214558571577072
Epoch 25, minibatch 16 of 16 -- Model loss: 0.1916632056236267
Epoch 26, minibatch 1 of 16 -- Model loss: 0.024723175913095474
Epoch 26, minibatch 2 of 16 -- Model loss: 0.034868475049734116
Epoch 26, minibatch 3 of 16 -- Model loss: 0.02544497139751911
Epoch 26, minibatch 4 of 16 -- Model loss: 0.028505517169833183
Epoch 26, minibatch 5 of 16 -- Model loss: 0.02903347834944725
Epoch 26, minibatch 6 of 16 -- Model loss: 0.03954000025987625
Epoch 26, minibatch 7 of 16 -- Model loss: 0.04748079925775528
Epoch 26, minibatch 8 of 16 -- Model loss: 0.030903775244951248
Epoch 26, minibatch 9 of 16 -- Model loss: 0.026273993775248528
Epoch 26, minibatch 10 of 16 -- Model loss: 0.045531682670116425
Epoch 26, minibatch 11 of 16 -- Model loss: 0.026711855083703995
Epoch 26, minibatch 12 of 16 -- Model loss: 0.03980354592204094
Epoch 26, minibatch 13 of 16 -- Model loss: 0.037662770599126816
Epoch 26, minibatch 14 of 16 -- Model loss

Epoch 34, minibatch 2 of 16 -- Model loss: 0.019932428374886513
Epoch 34, minibatch 3 of 16 -- Model loss: 0.01464405469596386
Epoch 34, minibatch 4 of 16 -- Model loss: 0.018249066546559334
Epoch 34, minibatch 5 of 16 -- Model loss: 0.013890162110328674
Epoch 34, minibatch 6 of 16 -- Model loss: 0.028423219919204712
Epoch 34, minibatch 7 of 16 -- Model loss: 0.038935158401727676
Epoch 34, minibatch 8 of 16 -- Model loss: 0.016780883073806763
Epoch 34, minibatch 9 of 16 -- Model loss: 0.014514394104480743
Epoch 34, minibatch 10 of 16 -- Model loss: 0.0384250208735466
Epoch 34, minibatch 11 of 16 -- Model loss: 0.030875762924551964
Epoch 34, minibatch 12 of 16 -- Model loss: 0.03314324468374252
Epoch 34, minibatch 13 of 16 -- Model loss: 0.018022645264863968
Epoch 34, minibatch 14 of 16 -- Model loss: 0.010915474034845829
Epoch 34, minibatch 15 of 16 -- Model loss: 0.06956348568201065
Epoch 34, minibatch 16 of 16 -- Model loss: 0.17803147435188293
Epoch 35, minibatch 1 of 16 -- Model lo

Epoch 42, minibatch 5 of 16 -- Model loss: 0.010180752724409103
Epoch 42, minibatch 6 of 16 -- Model loss: 0.025949135422706604
Epoch 42, minibatch 7 of 16 -- Model loss: 0.022371040657162666
Epoch 42, minibatch 8 of 16 -- Model loss: 0.01038768608123064
Epoch 42, minibatch 9 of 16 -- Model loss: 0.008074741810560226
Epoch 42, minibatch 10 of 16 -- Model loss: 0.024922708049416542
Epoch 42, minibatch 11 of 16 -- Model loss: 0.020311886444687843
Epoch 42, minibatch 12 of 16 -- Model loss: 0.02622726559638977
Epoch 42, minibatch 13 of 16 -- Model loss: 0.00511676911264658
Epoch 42, minibatch 14 of 16 -- Model loss: 0.006365691777318716
Epoch 42, minibatch 15 of 16 -- Model loss: 0.0547504797577858
Epoch 42, minibatch 16 of 16 -- Model loss: 0.14305762946605682
Epoch 43, minibatch 1 of 16 -- Model loss: 0.007153680548071861
Epoch 43, minibatch 2 of 16 -- Model loss: 0.006681537721306086
Epoch 43, minibatch 3 of 16 -- Model loss: 0.009851892478764057
Epoch 43, minibatch 4 of 16 -- Model lo

Epoch 50, minibatch 5 of 16 -- Model loss: 0.001579396310262382
Epoch 50, minibatch 6 of 16 -- Model loss: 0.0202022772282362
Epoch 50, minibatch 7 of 16 -- Model loss: 0.003990373574197292
Epoch 50, minibatch 8 of 16 -- Model loss: 0.0033045730087906122
Epoch 50, minibatch 9 of 16 -- Model loss: 0.0037733090575784445
Epoch 50, minibatch 10 of 16 -- Model loss: 0.01773349568247795
Epoch 50, minibatch 11 of 16 -- Model loss: 0.005761397536844015
Epoch 50, minibatch 12 of 16 -- Model loss: 0.022831402719020844
Epoch 50, minibatch 13 of 16 -- Model loss: 0.004126668442040682
Epoch 50, minibatch 14 of 16 -- Model loss: 0.0017098068492487073
Epoch 50, minibatch 15 of 16 -- Model loss: 0.051737524569034576
Epoch 50, minibatch 16 of 16 -- Model loss: 0.13132378458976746
Saved RNN net.
Saved dense net.


### 2.2.4 predicting user input sentence

In [50]:
# Enter sentences below:
#######################

user_inlist = [
    
    'we would rather have stayed home',
    'honestly, dont go',
    'this place is amazing'
]

In [51]:
# preparing user input dataset
# ----------------------------

xu,hu = prepare_use_input(user_inlist, num_layers, num_directions, hidden_state_size)

1. preparing user input..
2. Setting up datasets..
1. Itering through in dict..
2. Padding list..
##
torch.Size([6, 3, 300])
torch.FloatTensor
torch.Size([3, 1])
torch.FloatTensor
torch.Size([1, 3, 32])


In [52]:
# Inference ops
# -------------

pred = inference(xu,hu,'rnn_dense',md,use_cuda)

100%|██████████| 3/3 [00:00<00:00, 1042.75it/s]


In [53]:
# friendly display
# ----------------

for i in range(len(user_inlist)):
    print('Text: ' + user_inlist[i])
    print('Prediction: ' + str(round(pred[i,0]*100,2)) + ' % positive.')
    print('***\n')

Text: we would rather have stayed home
Prediction: 1.41 % positive.
***

Text: honestly, dont go
Prediction: 1.57 % positive.
***

Text: this place is amazing
Prediction: 56.19 % positive.
***



# Rough